In [7]:
# pip install "paddleocr>=2.0.1"

In [8]:
from paddleocr import PaddleOCR
# draw result
from PIL import Image, ImageDraw, ImageFont
import numpy as np
import math
import cv2
import string
import math
import pandas as pd

In [9]:
# Paddleocr supports Chinese, English, French, German, Korean, and Japanese.
# You can set the parameter `lang` as `ch`, `en`, `fr`, `german`, `korean`, `japan`
# to switch the language model in order.
ocr = PaddleOCR(use_angle_cls=True, lang='en')  # need to run only once to download and load the model into memory
img_path = './body.png'
result = ocr.ocr(img_path, cls=True)
for line in result:
    print(line)


[2024/01/18 18:44:08] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='C:\\Users\\19521/.paddleocr/whl\\det\\en\\en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='C:\\Users\\19521/.paddleocr/whl\\rec\\en\\en_PP-OCRv4_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_num=6, max_text_len

In [10]:
def resize_img(img, input_size=600):
    """
    resize img and limit the longest side of the image to input_size
    """
    img = np.array(img)
    im_shape = img.shape
    im_size_max = np.max(im_shape[0:2])
    im_scale = float(input_size) / float(im_size_max)
    img = cv2.resize(img, None, None, fx=im_scale, fy=im_scale)
    return img
def str_count(s):
    """
    Count the number of Chinese characters,
    a single English character, and a single digit
    equal to half the length of Chinese characters.
    
    Args:
        s (str): The input string.
        
    Returns:
        int: The number of Chinese characters.
    """
    count_zh = count_pu = 0
    s_len = len(s)
    en_dg_count = 0
    
    for c in s:
        if str(c) in string.ascii_letters or str(c).isdigit() or str(c).isspace():
            en_dg_count += 1
        elif str(c).isalpha():
            count_zh += 1
        else:
            count_pu += 1
    
    return s_len - math.ceil(en_dg_count / 2)


def draw_ocr(image, boxes, txts, scores):
    if scores is None:
        scores = [1] * len(boxes)

    img = np.array(image)
    arrayVal=[]
    for idx in range(len(boxes)):
        if scores[idx] < 0.5 or math.isnan(scores[idx]):
            continue
        coordinates = np.array(boxes[idx][0], dtype=np.int32).reshape(-1, 2)
        img = cv2.polylines(img, [coordinates], isClosed=True, color=(255, 0, 0), thickness=2)
        # img = resize_img(img, input_size=600)  # Placeholder for resize_img
    
    txt_img = []
    for index, txt in enumerate(txts):
        if txt is not None:
            arrayVal.append([index,txt,scores[index]])
    
    # Check if txt_img is not empty before concatenating
    if txt_img:
        txt_img_concatenated = np.concatenate(txt_img, axis=1)
        img = np.concatenate([img, txt_img_concatenated], axis=1)

    return img,arrayVal


# Your existing code
image = Image.open(img_path).convert('RGB')

boxes =[]
txts=[]
scores=[]
idx=0

for value in result[0]:
  scores.append(value[1][1])
  txts.append(value[1][0])
  boxes.append(value)
  idx+=1
# Set your desired drop_score value

drop_score = 0.5
im_show, arrayValue = draw_ocr(image, boxes, txts, scores)
im_show = Image.fromarray(im_show)
im_show.save('result.jpg')

# Create DataFrame from arrayValue
df = pd.DataFrame(arrayValue, columns=['Index', 'Value', 'Score'])
df.set_index('Index', inplace=True)  # Set 'Index' column as the index
df.to_csv('./result.csv')
